<a href="https://colab.research.google.com/github/MHusni1604/Pengantar-Anlisis-Data-Panel/blob/main/Tugas_Minggu_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis Dampak Trump Tariffs Terhadap Harga Saham LQ45
---
![Trump Tariffs](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Trump_showing_a_chart_with_reciprocal_tariffs_%28cropped%29.jpg/500px-Trump_showing_a_chart_with_reciprocal_tariffs_%28cropped%29.jpg)

# Pendahuluan

Pada tanggal 2 April 2025, pemerintahan Presiden Donald Trump mengumumkan kebijakan baru untuk tarif impor yang berdampak pada dinamika ekonomi global. Kebijakan ini dikenal sebagai "Trump Tariffs" yang secara signifikan menaikkan tarif rata-rata untuk barang impor ke Amerika Serikat. Tarif ini menyebabkan lonjakan tarif dari $2,5\%$ menjadi $27\%$ [[1](https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration)]. Kebijakan ini memicu gejolak pasar global termasuk tekananpada pasar keuangan negara berkembang seperti Indonesia.

Indeks LQ45 adalah indeks harga saham untuk 45 saham paling likuid di Bursa Efek Indonesia (BEI) yang menjadi indikator penting untuk menilai respon pasar domestik terhadap dinamika global [[2](https://id.wikipedia.org/wiki/Indeks_LQ45)]. Perubahan harga saham pada indeks ini dipengaruhi oleh sentimen internasional terhadap kondisi makro ekonomi secara umum. Oleh karena itu, dilakukan analisis pengaruh dari tarif trump terhadap indeks harga saham LQ45.

Penelitian ini bertujuan untuk mengukur dampak "Trump Tariffs" terhadap harga saham perusahaan-perusahaan pada indeks LQ45. Variabel dependen yang digunakan adalah harga penutupan saham harian, sementara variabel independen mencakup suku bunga acuan Bank Indonesia (BI rate), tingkat inflasi, nilai tukar Rupiah terhadap Dolar AS, serta variabel dummy yang menandai periode penerapan tarif. Dengan menggunakan pendekatan data panel yang menggabungkan dimensi lintas-perusahaan dan lintas-waktu, analisis ini diharapkan mampu menangkap dinamika yang kompleks dalam respons harga saham terhadap tekanan eksternal dan kondisi makro domestik [[3](https://doi.org/10.24912/jmk.v4i1.17118)].

# Metodologi

## Sumber Data
Penelitian ini menggunakan pendekatan kuantitatif dengan metode regresi data panel untuk menganalisis pengaruh kebijakan tarif impor Amerika Serikat tahun 2025 (Tarif Trump 2025) terhadap harga saham perusahaan yang tergabung dalam Indeks LQ45 di Bursa Efek Indonesia. Data yang digunakan merupakan data sekunder yang diperoleh dari dua sumber utama, yaitu [Yahoo Finance](finance.yahoo.com) untuk data harga penutupan saham harian dan nilai tukar Rupiah terhadap Dolar AS, serta [situs resmi Bank Indonesia](bi.go.id) untuk data suku bunga acuan (BI Rate) dan tingkat inflasi.

## Spesifikasi Model
Penelitian ini mengaplikasikan Fixed Effect Model (FEM) dengan pendekatan entity effect, yaitu mempertimbangkan adanya perbedaan karakteristik tetap (time-invariant) antar perusahaan yang dapat memengaruhi harga saham. Model ini lebih tepat digunakan ketika diasumsikan bahwa masing-masing perusahaan memiliki efek khusus yang tidak diamati tetapi tetap dari waktu ke waktu (seperti reputasi, manajemen internal, atau struktur industri).

Model regresi panel dengan efek tetap entitas dapat dirumuskan sebagai:
$$Y_{it} = \alpha_{i} + \beta_{1}(\text{BI-Rate})_{t} + \beta_{2}(\text{Rate Inflasi})_{t} + \beta_{3}(\text{Kurs})_{t} + \beta_{4}(\text{Dummy Tarif})_{t} + \epsilon_{it}$$
Keterangan:
- $Y_{it}$: Harga penutupan saham erusahaan $i$ pada waktu $t$
- $\alpha_{i}$: Intersep khusus untuk masing-masing perusahaan
- $\beta_{1},...,\beta_{4}$: Koefisien regresi
- $\epsilon_{it}$: Galat

Dengan pendekatan ini, variabel-variabel independen yang bersifat makro (BI Rate, inflasi, kurs dolar, dan dummy tarif Trump) dianalisis dalam pengaruhnya terhadap harga saham, dengan memperhitungkan heterogenitas perusahaan dalam LQ45. Model ini akan mengeliminasi bias dari variabel-variabel yang tidak diamati tetapi konstan dalam perusahaan.

# Hasil dan Pembahasan

## Import Library

In [17]:
pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.3 MB/s eta 0:00:00


In [18]:
# Library umum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pengambilan data finansial
import yfinance as yf

# Statistik dan ekonometrika
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

# Model panel data
from linearmodels.panel import PanelOLS

# Machine learning preprocessing dan reduksi dimensi
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Data Loading

Mengambil data saham LQ45 dari laman Yahoo Finance

In [3]:
tickers = [
    "ACES.JK", "ADMR.JK", "ADRO.JK", "AKRA.JK", "AMMN.JK", "AMRT.JK", "ANTM.JK", "ARTO.JK",
    "ASII.JK", "BBCA.JK", "BBNI.JK", "BBRI.JK", "BBTN.JK", "BMRI.JK", "BRIS.JK", "BRPT.JK",
    "CPIN.JK", "CTRA.JK", "ESSA.JK", "EXCL.JK", "GOTO.JK", "ICBP.JK", "INCO.JK", "INDF.JK",
    "INKP.JK", "ISAT.JK", "ITMG.JK", "JPFA.JK", "JSMR.JK", "KLBF.JK", "MAPA.JK", "MAPI.JK",
    "MBMA.JK", "MDKA.JK", "MEDC.JK", "PGAS.JK", "PGEO.JK", "PTBA.JK", "SIDO.JK", "SMGR.JK",
    "SMRA.JK", "TLKM.JK", "TOWR.JK", "UNTR.JK", "UNVR.JK"
]

data = yf.download(tickers, start="2025-01-01", end="2025-05-31")['Close']

data_panel = data.stack().reset_index()
data_panel.columns = ['Date', 'Ticker', 'Close']

df_saham = data_panel.copy()
df_saham['Date'] = pd.to_datetime(df_saham['Date'])

df_saham

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  45 of 45 completed


,Date,Ticker,Close
0,2025-01-02,ACES.JK,755.000000
1,2025-01-02,ADMR.JK,1165.000000
2,2025-01-02,ADRO.JK,2530.000000
3,2025-01-02,AKRA.JK,1110.232544
4,2025-01-02,AMMN.JK,8475.000000
...,...,...,...
4090,2025-05-28,SMRA.JK,404.000000
4091,2025-05-28,TLKM.JK,2820.000000
4092,2025-05-28,TOWR.JK,585.000000
4093,2025-05-28,UNTR.JK,22450.000000


Mengambil data BI-rate dari laman resmi BI

In [4]:
df_birate = {
    'Date' : ['2024-12-18', '2025-01-15', '2025-02-19', '2025-03-19', '2025-04-23', '2025-05-21'],
    'BI-Rate' : [0.0600, 0.0575, 0.0575, 0.0575, 0.0575, 0.0550]
}

df_birate = pd.DataFrame(df_birate)
df_birate['Date'] = pd.to_datetime(df_birate['Date'])
df_birate['Start Date'] = df_birate['Date']
df_birate['End Date'] = df_birate['Date'].shift(-1)

last_end_date = df_saham['Date'].max()
df_birate.loc[df_birate.index[-1], 'End Date'] = last_end_date

df_birate

,Date,BI-Rate,Start Date,End Date
0,2024-12-18,0.0600,2024-12-18,2025-01-15
1,2025-01-15,0.0575,2025-01-15,2025-02-19
2,2025-02-19,0.0575,2025-02-19,2025-03-19
3,2025-03-19,0.0575,2025-03-19,2025-04-23
4,2025-04-23,0.0575,2025-04-23,2025-05-21
5,2025-05-21,0.0550,2025-05-21,2025-05-28


In [5]:
wide_df = df_saham.reset_index().pivot(index='Date', columns='Ticker', values='Close').reset_index()

def assign_value(date):
    for i in range(len(df_birate)):
        if df_birate.loc[i, 'Start Date'] <= date <= df_birate.loc[i, 'End Date']:
            return df_birate.loc[i, 'BI-Rate']
    return None

wide_df['BI-Rate'] = wide_df['Date'].apply(assign_value)
df_saham = wide_df.melt(id_vars=['Date', 'BI-Rate'], var_name='Ticker', value_name='Close')

df_saham

Mengambil data tingkat inflasi dari laman resmi BI

In [7]:
df_inflasi = {
    'Date' : ['2025-01-01', '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01'],
    'Rate Inflasi' : [0.0160, 0.0195, 0.0103, -0.0009, 0.0076]
}

df_inflasi = pd.DataFrame(df_inflasi)
df_inflasi['Date'] = pd.to_datetime(df_inflasi['Date'])

df_inflasi

,Date,Rate Inflasi
0,2025-01-01,0.0160
1,2025-02-01,0.0195
2,2025-03-01,0.0103
3,2025-04-01,-0.0009
4,2025-05-01,0.0076


In [8]:
df_saham['Month'] = df_saham['Date'].dt.month
df_inflasi['Month'] = df_inflasi['Date'].dt.month
df_saham = df_saham.merge(df_inflasi[['Month', 'Rate Inflasi']], on='Month', how='left')
df_saham.drop(columns=['Month'], inplace=True)

Mengambil data rate nilai tukar dollar dari laman Yahoo Finance

In [9]:
kurs = yf.download(
    "USDIDR=X",
    start=wide_df['Date'].min().strftime('%Y-%m-%d'),
    end=(wide_df['Date'].max() + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
)

df_kurs = kurs[['Close']].reset_index()
df_kurs.columns = ['Date', 'USD_IDR']

df_kurs

[*********************100%***********************]  1 of 1 completed


,Date,USD_IDR
0,2025-01-02,16230.500000
1,2025-01-03,16201.099609
2,2025-01-06,16178.799805
3,2025-01-07,16179.099609
4,2025-01-08,16157.700195
...,...,...
98,2025-05-22,16347.599609
99,2025-05-23,16356.900391
100,2025-05-26,16211.000000
101,2025-05-27,16243.500000


In [10]:
df_saham = df_saham.merge(df_kurs, left_on='Date', right_on='Date', how='left')
df_saham

,Date,BI-Rate,Ticker,Close,Rate Inflasi,USD_IDR
0,2025-01-02,0.060,ACES.JK,755.0,0.0160,16230.500000
1,2025-01-03,0.060,ACES.JK,755.0,0.0160,16201.099609
2,2025-01-06,0.060,ACES.JK,755.0,0.0160,16178.799805
3,2025-01-07,0.060,ACES.JK,735.0,0.0160,16179.099609
4,2025-01-08,0.060,ACES.JK,705.0,0.0160,16157.700195
...,...,...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0,0.0076,16347.599609
4091,2025-05-23,0.055,UNVR.JK,1785.0,0.0076,16356.900391
4092,2025-05-26,0.055,UNVR.JK,1765.0,0.0076,16211.000000
4093,2025-05-27,0.055,UNVR.JK,1740.0,0.0076,16243.500000


In [11]:
df_saham['Trump Tariffs'] = (df_saham.Date >= pd.to_datetime('2025-04-05')).astype(int)
df_saham

,Date,BI-Rate,Ticker,Close,Rate Inflasi,USD_IDR,Trump Tariffs
0,2025-01-02,0.060,ACES.JK,755.0,0.0160,16230.500000,0
1,2025-01-03,0.060,ACES.JK,755.0,0.0160,16201.099609,0
2,2025-01-06,0.060,ACES.JK,755.0,0.0160,16178.799805,0
3,2025-01-07,0.060,ACES.JK,735.0,0.0160,16179.099609,0
4,2025-01-08,0.060,ACES.JK,705.0,0.0160,16157.700195,0
...,...,...,...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0,0.0076,16347.599609,1
4091,2025-05-23,0.055,UNVR.JK,1785.0,0.0076,16356.900391,1
4092,2025-05-26,0.055,UNVR.JK,1765.0,0.0076,16211.000000,1
4093,2025-05-27,0.055,UNVR.JK,1740.0,0.0076,16243.500000,1


In [12]:
df_saham.sort_values(by=['Ticker', 'Date'], inplace=True)
df_saham.set_index(['Ticker', 'Date'], inplace=True)
df_saham

BI-Rate   Close  Rate Inflasi       USD_IDR  Trump Tariffs
Ticker  Date                                                                  
ACES.JK 2025-01-02    0.060   755.0        0.0160  16230.500000              0
        2025-01-03    0.060   755.0        0.0160  16201.099609              0
        2025-01-06    0.060   755.0        0.0160  16178.799805              0
        2025-01-07    0.060   735.0        0.0160  16179.099609              0
        2025-01-08    0.060   705.0        0.0160  16157.700195              0
...                     ...     ...           ...           ...            ...
UNVR.JK 2025-05-22    0.055  1815.0        0.0076  16347.599609              1
        2025-05-23    0.055  1785.0        0.0076  16356.900391              1
        2025-05-26    0.055  1765.0        0.0076  16211.000000              1
        2025-05-27    0.055  1740.0        0.0076  16243.500000              1
        2025-05-28    0.055  1730.0        0.0076  16279.000000              1

[4095 rows x 5 columns]

## Model Development

Data Lengkap setelah disatukan

In [13]:
df_saham.dropna(inplace=True)
df_saham

BI-Rate   Close  Rate Inflasi       USD_IDR  Trump Tariffs
Ticker  Date                                                                  
ACES.JK 2025-01-02    0.060   755.0        0.0160  16230.500000              0
        2025-01-03    0.060   755.0        0.0160  16201.099609              0
        2025-01-06    0.060   755.0        0.0160  16178.799805              0
        2025-01-07    0.060   735.0        0.0160  16179.099609              0
        2025-01-08    0.060   705.0        0.0160  16157.700195              0
...                     ...     ...           ...           ...            ...
UNVR.JK 2025-05-22    0.055  1815.0        0.0076  16347.599609              1
        2025-05-23    0.055  1785.0        0.0076  16356.900391              1
        2025-05-26    0.055  1765.0        0.0076  16211.000000              1
        2025-05-27    0.055  1740.0        0.0076  16243.500000              1
        2025-05-28    0.055  1730.0        0.0076  16279.000000              1

[4050 rows x 5 columns]

Ringakasan dari data yang digunakan

In [19]:
df_saham.describe()

,BI-Rate,Close,Rate Inflasi,USD_IDR,Trump Tariffs
count,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000
mean,0.057639,3531.395472,0.011171,16430.656630,0.355556
std,0.001011,4968.159705,0.006871,208.341001,0.478740
min,0.055000,64.000000,-0.000900,16157.700195,0.000000
25%,0.057500,945.209259,0.007600,16279.000000,0.000000
50%,0.057500,1895.000000,0.010300,16352.250000,0.000000
75%,0.057500,4000.000000,0.016000,16538.300781,1.000000
max,0.060000,26425.000000,0.019500,17051.900391,1.000000


Cek multikolinearitas dari variabel


In [14]:
X = df_saham.drop(columns=['Close'])
X = add_constant(X)

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data.drop(index=0)

,Variable,VIF
1,BI-Rate,1.196004
2,Rate Inflasi,5.335810
3,USD_IDR,3.175236
4,Trump Tariffs,3.328826


Hasil uji multikolinearitas menggunakan Variance Inflation Factor (VIF) menunjukkan bahwa seluruh variabel independen dalam model memiliki nilai VIF di bawah ambang batas umum sebesar $10$. Nilai VIF tertinggi terdapat pada variabel `Rate Inflasi` sebesar $5,34$, sementara variabel lainnya berada dalam kisaran $1,20$ hingga $3,33$. Hal ini mengindikasikan bahwa tidak terdapat masalah multikolinearitas serius antar variabel independen dalam model, sehingga model regresi layak untuk diinterpretasikan lebih lanjut.

In [21]:
df_saham['const'] = 1

mod = PanelOLS(
    dependent=df_saham['Close'],
    exog=df_saham[['const', 'BI-Rate', 'Rate Inflasi', 'USD_IDR', 'Trump Tariffs']],
    entity_effects=True,
)

res = mod.fit()
print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  Close   R-squared:                        0.1390
Estimator:                   PanelOLS   R-squared (Between):           -2.22e-16
No. Observations:                4050   R-squared (Within):               0.1390
Date:                Thu, Jun 05 2025   R-squared (Overall):              0.0010
Time:                        15:26:25   Log-likelihood                -2.996e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      161.52
Entities:                          45   P-value                           0.0000
Avg Obs:                       90.000   Distribution:                  F(4,4001)
Min Obs:                       90.000                                           
Max Obs:                       90.000   F-statistic (robust):             161.52
                            

Model panel fixed effect ini menguji pengaruh variabel makroekonomi dan kebijakan tarif Trump terhadap harga penutupan saham perusahaan dalam indeks LQ45. Nilai R-squared (Within) sebesar $0,1390$ menunjukkan bahwa sekitar $13,9\%$ variasi harga saham dalam perusahaan dari waktu ke waktu dapat dijelaskan oleh variabel-variabel independen dalam model. Sisanya dijelaskan oleh faktor lain di luar model.

Hasil F-statistic sebesar $161,52$ dengan p-value $0,0000$ mengindikasikan bahwa model secara keseluruhan signifikan secara statistik pada level kepercayaan $99\%$, artinya setidaknya satu dari variabel independen secara signifikan memengaruhi harga saham.

Uji poolability (F-test for Poolability) juga signifikan (p-value = $0,0000$), yang menguatkan bahwa penggunaan fixed effect (entity effect) lebih tepat dibanding pooled OLS.

In [22]:
residuals = res.resids
exog = df_saham[['const', 'BI-Rate', 'Rate Inflasi', 'USD_IDR', 'Trump Tariffs']]

bp_test = het_breuschpagan(residuals, exog)
print('Breusch-Pagan test statistic:', bp_test[0])
print('p-value:', bp_test[1])

Breusch-Pagan test statistic: 6.949578848639282
p-value: 0.13857686439481584


Hasil uji Breusch-Pagan menunjukkan nilai statistik sebesar $6,95$ dengan p-value sebesar $0,139$. Karena p-value lebih besar dari tingkat signifikansi umum $0,05$, maka tidak ada bukti yang cukup untuk menolak hipotesis nol yang menyatakan bahwa varians residual adalah homogen (tidak terjadi heteroskedastisitas). Dengan kata lain, model regresi yang digunakan tidak mengalami masalah heteroskedastisitas, sehingga asumsi varians residual konstan terpenuhi.

In [23]:
dw_stat = durbin_watson(res.resids)
print('Durbin-Watson statistic:', dw_stat)

Durbin-Watson statistic: 0.18233818350062683


Nilai Durbin-Watson sebesar $0,18$ menunjukkan adanya autokorelasi positif yang kuat pada residual model. Idealnya, nilai Durbin-Watson berkisar antara $1,5$ hingga $2,5$ yang menandakan tidak ada autokorelasi. Nilai yang sangat rendah seperti ini mengindikasikan bahwa error residual cenderung berurutan dan saling terkait.

# Kesimpulan

Berdasarkan estimasi model fixed effect per entitas, diperoleh bahwa variabel BI Rate, tingkat inflasi, nilai tukar USD/IDR, dan periode tarif Trump 2025 secara simultan berpengaruh signifikan terhadap harga penutupan saham pada perusahaan-perusahaan di Indeks LQ45. Koefisien positif pada BI Rate ($+56,680$), inflasi ($+18,590$), dan dummy tarif Trump ($+333,05$) mengindikasikan bahwa kenaikan suku bunga, inflasi, maupun diberlakukannya tarif cenderung meningkatkan harga saham LQ45. Sebaliknya, depresiasi Rupiah terhadap Dolar AS (USD/IDR) menunjukkan efek negatif sebesar $–0,573$ per Rp1, yang konsisten dengan peningkatan biaya impor dan tekanan terhadap profitabilitas perusahaan. Nilai R-squared within sebesar $13,9 \%$ menandakan model mampu menjelaskan sebagian variasi pergerakan harga saham dari waktu ke waktu, meski sebagian besar variasi masih dijelaskan faktor lain di luar model.

Hasil uji Breusch-Pagan menunjukkan tidak adanya heteroskedastisitas, sedangkan Durbin-Watson mengindikasikan adanya autokorelasi positif yang cukup tinggi pada residual. Untuk memperbaiki isu autokorelasi, dapat dipertimbangkan penggunaan robust standard errors atau spesifikasi model alternatif yang menangani serial correlation. Dari sudut kebijakan, temuan bahwa periode tarif Trump 2025 membawa efek positif pada harga saham LQ45 mengindikasikan bahwa pasar domestik mungkin melihat peluang diversifikasi sumber pertumbuhan ketika ketegangan perdagangan internasional meningkat. Namun, kenaikan inflasi dan suku bunga juga berimplikasi pada lonjakan harga saham, yang perlu diwaspadai oleh investor karena bisa mencerminkan risiko biaya produksi yang meningkat.

Secara keseluruhan, penelitian ini mengonfirmasi bahwa kebijakan eksternal (khususnya tarif impor Amerika Serikat) berdampak nyata pada pasar modal Indonesia melalui kanal makroekonomi. Ke depan, disarankan untuk meneliti lebih lanjut faktor-faktor lain seperti ekspektasi pasar, likuiditas sektor, atau kondisi keuangan perusahaan yang mungkin turut memengaruhi harga saham, agar model mampu menjelaskan variasi yang lebih besar dan memberikan insight kebijakan yang lebih komprehensif.

# Sumber

[[1](https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration)] "Tariffs in the second Trump administration", Wikipedia, [Online]. Tersedia: https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration. [Diakses: Jun. 5, 2025]. \
[[2](https://id.wikipedia.org/wiki/Indeks_LQ45)] "Indeks LQ45", Wikipedia, [Online]. Tersedia: https://id.wikipedia.org/wiki/Indeks_LQ45. [Diakses: Jun. 5, 2025]. \
[[3](https://doi.org/10.24912/jmk.v4i1.17118)] A. B. Wijaya dan N. M. Nuryasman, "Pengaruh Variabel Makroekonomi dan Rasio Keuangan terhadap Harga Saham," Jurnal Manajerial dan Kewirausahaan, vol. 4, no. 1, pp. 12–21, Jan. 2022, doi: 10.24912/jmk.v4i1.17118. \
[4] E. W. Frees, Longitudinal and Panel Data: Analysis and Applications in the Social Sciences. Cambridge: Cambridge University Press, 2004.